In [ ]:
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from trl import SFTConfig, SFTTrainer

from deepfabric import DeepFabricCallback
from deepfabric.evaluation import Evaluator, EvaluatorConfig, InferenceConfig


In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")


In [ ]:
peft_config = LoraConfig(
    r=8,              # Drop from 16
    lora_alpha=16,    # Keep 2x ratio
    lora_dropout=0.1, # Bump from 0.05
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

In [ ]:

dataset = load_dataset("alwaysfurther/coding-test-dataset", split="train")
split = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = split['train']  # ~88 samples for training
eval_ds = split['test']    # ~10 samples for evaluation

print(f"Train: {len(train_ds)}, Eval: {len(eval_ds)}")
# With 1k samples: ~800, ~100, ~100


def format_example(example):
    messages = [{k: v for (k, v) in msg.items() if v is not None} for msg in example['messages']]
    tools = example.get('tools', None)  # Get tools from the sample
    return {'text': tokenizer.apply_chat_template(
        messages,
        tools=tools,
        tokenize=False
    )}

train_dataset = train_ds.map(format_example)

In [ ]:
training_args = SFTConfig(
    output_dir="./lora-output",
    max_steps=5,
    learning_rate=2e-4,
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    processing_class=tokenizer,
)

trainer.add_callback(DeepFabricCallback(trainer))

trainer.train()

In [ ]:
# Configure the evaluator
config = EvaluatorConfig(
    inference_config=InferenceConfig(
        model="Qwen/Qwen2.5-1.5B-Instruct",
        backend="transformers",
        temperature=0.1,
        max_tokens=2048,
    ),
    max_samples=100,
    save_predictions=True,
    multi_turn=False,
    output_path="./eval_trained_results.json",
)

# Create evaluator and check if PEFT adapter loaded
evaluator = Evaluator(config)
results = evaluator.evaluate(dataset=eval_ds)

# Print summary
evaluator.print_summary(results.metrics)

# Cleanup GPU memory
evaluator.cleanup()
